# Advent of Code Day 24
#### Problem in full can be found here: https://adventofcode.com/2023/day/24

In [81]:
# Given an input file where each line consists of 3 integers to represent x, y, z followed by an @
# which follows another 3 integers to represent the slopes dx, dy, dz

# Get the input from file and store in data array
file = open("inputday24.txt")
data = []
for line in file:
    line = line.strip()
    parts = line.split("@")
    position = parts[0].split(",")
    velocity = parts[1].split(",")
    data.append([position[0], position[1], position[2], velocity[0], velocity[1], velocity[2]])
file.close()

# Part 1
# Given a bound (the intersection has to fall within this bound), using only the x and y from the input,
# loop through each pair of lines and see if they intersect wiithin the the test bounds. Print the sum of
# pairs that meet the criteria

# Set test bounds
testMin = 200000000000000
testMax = 400000000000000

# Loop through all pairs
sum = 0
for i in range(len(data)-1):
    for j in range(i+1, len(data)):
        # Find the intersection
        x, y = findIntersection(data[i], data[j])
        # If an intersection exists and is within the test bounds, add 1 to sum
        if x and y and testMin <= x <= testMax and testMin <= y <= testMax:
            sum += 1
print(sum)

# Part 2
# Find a point on the map that will intersect with every line in the data and print the sum of x, y, z of
# that point
print(findPosition(data))

17867
557743507346379


In [78]:
# A functino that finds the intersection by calculating the slopes of both lines
def findIntersection(hailstone1, hailstone2):
    # Set the variables required
    x1, x2 = int(hailstone1[0]), int(hailstone2[0])
    y1, y2 = int(hailstone1[1]), int(hailstone2[1])
    dx1, dx2 = int(hailstone1[3]), int(hailstone2[3])
    dy1, dy2 = int(hailstone1[4]), int(hailstone2[4])

    # Calculate the slope
    slope1 = dy1 / dx1
    slope2 = dy2 / dx2
    # IF slopes are equal, return none because it means its parallel
    if slope1 == slope2:
        return None, None

    # Calculate intersection from y = mx + b
    x = (y1 - y2 + slope2 * x2 - slope1 * x1) / (slope2 - slope1)
    y = slope1 * (x - x1) + y1

    # verify that the intersection is possible to get to with the slope (not in the past)
    if (np.sign(x - x1) != np.sign(dx1) or 
        np.sign(x - x2) != np.sign(dx2) or 
        np.sign(y - y1) != np.sign(dy1) or 
        np.sign(y - y2) != np.sign(dy2)):
        return None, None

    # Return the intersection
    return (x,y)

In [79]:
from z3 import *

def findPosition(data):
    # Create Z3 variables
    x, y, z, dx, dy, dz = map(z3.Int, ["x", "y", "z", "dx", "dy", "dz"])

    # Create Z3 solver
    solver = Solver()

    # Loop through each point in the data and add the line to the solver
    for i in range(len(data)):
        x1 = int(data[i][0])
        y1 = int(data[i][1])
        z1 = int(data[i][2])
        dx1 = int(data[i][3])
        dy1 = int(data[i][4])
        dz1 = int(data[i][5])
        # t = a variable to distinct between the different lines
        t = z3.Int(f"t{i}")
        solver.add(x + dx*t == x1+dx1*t)
        solver.add(y + dy*t == y1+dy1*t)
        solver.add(z + dz*t == z1+dz1*t)

    # Check for a solution and return x + y + z if there is one, 0 otherwise
    if solver.check() == sat:
        return solver.model().eval(x+y+z)
    else:
        return 0